In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_context("talk")
%matplotlib inline

import plotly.offline as py
py.init_notebook_mode(connected=True)

In [ ]:
labels = ("glioma", "meningioma", "notumor", "pituitary")
img_dim = (512, 512)

In [ ]:
import numpy as np
import os
import cv2

def load_images_dataset(label, num_images, kernal_size, img_dim):
    root = os.path.join(os.getcwd(), 'data') # get system path to data folder
    path = os.path.join(root, label) # get path to label folder
    file_paths = os.listdir(path) # list all files

    random_select = np.random.choice(file_paths, num_images, replace=False) # get N random selection of files
    images = []
    for f in random_select:                                                 # Preprocess images
        img_path = os.path.join(path, f)                                    # Get full file path
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)                    # Read image as grayscale
        img = cv2.resize(img, img_dim, interpolation=cv2.INTER_LINEAR)      # Resize image to correct dimensions
        ksize = (kernal_size, kernal_size)
        blur = cv2.blur(img, ksize)                                         # Blur image with square kernel size given
        normalized_img = cv2.normalize(blur, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX)    # normalize image
        images.append(normalized_img)

    return images

In [ ]:
images = load_images_dataset('glioma', 200, 5, img_dim)
plt.imshow(images[1], cmap='gray')

In [ ]:
def convert_images_to_np_array(img_list):
    size = len(img_list)
    num_col = img_list[0].shape[0] * img_list[0].shape[1]   # get number of cols by multiplying dims of first image, all same dim

    for i in range(size):
        img_list[i] =  np.ndarray.flatten(img_list[i]).reshape(num_col, 1)      # flatten image

    img_stack = np.dstack(img_list)                                             # Remove extra axis
    img_stack = np.rollaxis(img_stack, axis = 2, start = 0)
    img_stack = img_stack.reshape(size, num_col)

    return img_stack


In [ ]:
from sklearn.utils import shuffle
def create_dataframe(num_images, kernal_size, img_dim):

    df_list = []

    for l in labels:    # for each label
        temp_list = load_images_dataset(l, num_images, kernal_size, img_dim)    # load images
        temp_array = convert_images_to_np_array(temp_list)                      # convert images to np arraies
        temp_df = pd.DataFrame(temp_array)                                      # convert each array to a df
        temp_df['label'] = l                                                    # add corresponding label to group of images
        df_list.append(temp_df)

    act = pd.concat(df_list, ignore_index=True)         # combine dataframes into one
    final = shuffle(act).reset_index()                  # shuffle images
    final = final.drop(columns=['index'], axis=1)       # drop unneeded index col

    return final

In [ ]:
df = create_dataframe(num_images=200, kernal_size=5, img_dim=img_dim)
print(df.shape)
df.head()
df.tail()

In [ ]:
from sklearn.preprocessing import StandardScaler


def split_x_and_y(df):

    x = df.iloc[:, :-1] # get image data from data frame
    y = df.iloc[:, -1]  # get label data from data frame

    scaler = StandardScaler()
    x_scaled = scaler.fit_transform(x)

    return (x_scaled, y)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import classification_report, r2_score

# Split features and labels
x, y = split_x_and_y(df)

# Apply K-means clustering with 4 clusters (one for each tumor type)
kmeans_model = KMeans(n_clusters=4, random_state=42, max_iter=300)
kmeans_model.fit(x)

# Get cluster predictions
y_pred_clusters = kmeans_model.predict(x)

# Map clusters to actual labels by finding the most common label in each cluster
cluster_to_label = {}
for cluster_id in range(4):
    mask = y_pred_clusters == cluster_id
    most_common_label = y[mask].mode()[0]
    cluster_to_label[cluster_id] = most_common_label

# Map predictions to actual label names
y_pred_mapped = [cluster_to_label[cluster] for cluster in y_pred_clusters]

# Calculate accuracy
accuracy = metrics.accuracy_score(y, y_pred_mapped)
print(f"K-means clustering accuracy: {accuracy * 100:.2f}%")